# Example: Multi-layer Perceptron on MNIST

In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
from chainer import (Chain, ChainList, Function, Variable, datasets,
                     gradient_check, iterators, link, optimizers, report,
                     serializers, training, utils)
from chainer.training import extensions

In [4]:
# obtain mnist datasets
train, test = datasets.get_mnist()
train, test

(<chainer.datasets.tuple_dataset.TupleDataset at 0x7f99200fd9b0>,
 <chainer.datasets.tuple_dataset.TupleDataset at 0x7f98f316cc18>)

In [5]:
# make iterator
train_iter = iterators.SerialIterator(train, batch_size=100, shuffle=True)
test_iter = iterators.SerialIterator(
    test, batch_size=100, shuffle=False, repeat=False)

In [6]:
# define model
class MLP(Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of input to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(x))
        logits = self.l3(h2)
        return logits

In [7]:
class Classifier(Chain):
    def __init__(self, predictor):
        """
        predictor: chain
           this is model
        """
        super(Classifier, self).__init__()
        with self.init_scope():
            self.predictor = predictor
            
    def __call__(self, x, t):
        """
        x, t: variable
           these are input data
        """
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)  # define loss function
        accuracy = F.accuracy(y, t)  # calculate metrics
        report({'loss': loss, 'accuracy': accuracy}, self)  # report loss and accuracy to trainer
        return loss

In [8]:
model = L.Classifier(MLP(100, 10))  # the input size, 784, is infferd
optimizer = optimizers.SGD()  # define optimizer
optimizer.setup(model)
# optimizer.add_hook(chainer.optimizer.WeightDecay(0.00005))

In [9]:
updater = training.StandardUpdater(train_iter, optimizer)  # forward/backward coumputation & Parametor update
trainer = training.Trainer(updater, (20, "epoch"), out="result")  # whole procedure

In [11]:
# evaluate model at end of every epoch
trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.LogReport())  # output repoted value
trainer.extend(extensions.PrintReport(
    ["epoch", "main/accuracy", "validation/main/accuracy"]))  # Print logreport
trainer.extend(extensions.ProgressBar())  # show the progress bar

`PrintReport()`の引数の名前は`L.Classifier`のなかで`chiner.report`がつかわれているので，
引数の名前指定には`chainer.report`が関係してくる?

https://docs.chainer.org/en/stable/reference/util/generated/chainer.report.html?highlight=report

In [12]:
trainer.run()  # 20 epoch だけ訓練差させる

epoch       main/accuracy  validation/main/accuracy
     total [..................................................]  0.83%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    107.16 iters/sec. Estimated time to finish: 0:01:50.116686.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 20 epochs
    72.657 iters/sec. Estimated time to finish: 0:02:41.030514.
     total [#.................................................]  3.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 20 epochs
    75.071 iters/sec. Estimated time to finish:

     total [#############.....................................] 27.50%
this epoch [#########################.........................] 50.00%
      3300 iter, 5 epoch / 20 epochs
    72.322 iters/sec. Estimated time to finish: 0:02:00.295254.
     total [##############....................................] 28.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 20 epochs
     73.11 iters/sec. Estimated time to finish: 0:01:57.631642.
     total [##############....................................] 29.17%
this epoch [#########################################.........] 83.33%
      3500 iter, 5 epoch / 20 epochs
    72.561 iters/sec. Estimated time to finish: 0:01:57.142769.
6           0.9073         0.9147                    
     total [###############...................................] 30.00%
this epoch [..................................................]  0.00%
      3600 iter, 6 epoch / 20 epochs
    71.606 iters/sec. Estimated time t

     total [###########################.......................] 54.17%
this epoch [#########################################.........] 83.33%
      6500 iter, 10 epoch / 20 epochs
    63.492 iters/sec. Estimated time to finish: 0:01:26.624555.
11          0.922333       0.9275                    
     total [###########################.......................] 55.00%
this epoch [..................................................]  0.00%
      6600 iter, 11 epoch / 20 epochs
    63.183 iters/sec. Estimated time to finish: 0:01:25.466277.
     total [###########################.......................] 55.83%
this epoch [########..........................................] 16.67%
      6700 iter, 11 epoch / 20 epochs
    63.135 iters/sec. Estimated time to finish: 0:01:23.947127.
     total [############################......................] 56.67%
this epoch [################..................................] 33.33%
      6800 iter, 11 epoch / 20 epochs
    63.563 iters/sec. Estimated ti

     total [########################################..........] 80.83%
this epoch [########..........................................] 16.67%
      9700 iter, 16 epoch / 20 epochs
    61.184 iters/sec. Estimated time to finish: 0:00:37.591420.
     total [########################################..........] 81.67%
this epoch [################..................................] 33.33%
      9800 iter, 16 epoch / 20 epochs
    61.052 iters/sec. Estimated time to finish: 0:00:36.034690.
     total [#########################################.........] 82.50%
this epoch [#########################.........................] 50.00%
      9900 iter, 16 epoch / 20 epochs
    60.963 iters/sec. Estimated time to finish: 0:00:34.446944.
     total [#########################################.........] 83.33%
this epoch [#################################.................] 66.67%
     10000 iter, 16 epoch / 20 epochs
    60.882 iters/sec. Estimated time to finish: 0:00:32.850312.
     total [############